# ML Pipeline

My goals for this dataset was to extract meaningful features and classify the flight.

## Model Selection

### Supervised vs Unsuperivsed Learning

The first question I asked myself was should I use supervised or unsupervised techinques. To a large extent I like the certaininty of supervised techinque. There are verifiable model metrics that explicitly inform of me my success or failure. But would slapping on some labled really help me understand the true nature of the data? Below is my thought process:

Key Advantages of Supervised Learning:

- Results are directly interpretable since each class has a known real-world meaning
- Can leverage existing domain expertise through the labeling process
- Performance can be quantitatively validated against ground truth

Key Advantages of Unsupervised Learning:

- Can discover novel patterns that weren't previously known or considered
- Can analyze the complete dataset without being limited by label availability
- Not limited by the potential quality issues or obtuseness of labels

An unsupervised learning approach lets the data tell its own story. Rather than forcing the data into predefined boxes, we're letting the model make its own findings, discovering the true nature of contend of the data in ADSB records.

### Model Selection

Having decided to use unsupervised techinques, I now needed to choose which one(s). I really wanted something that would detect subtle variations in flight traces and that would not only be able to truly process high level data aggregations. At the same time, I want something that was performant or at least could be well optimized given the right hardware. My first priority though was really getting choosing a model that would get into the data, and if the outputs would be that much better I would be willing to make some performance trade-offs. Even though this is an academic project, I always keep model performance in mind because I want to get the most experience possible with models that make sense to deploy on real world problems ata production scale.

Here were the options I considered:

Traditional Machine Learning Models

- Traditional Clustering (K-means/DBSCAN)
  - Pros:
    - Computationally efficient and easy to implement
    - Very interpretable results with clear cluster centroids
    - Minimal hyperparameter tuning needed compared to deep learning approaches
  - Cons:
    - Cannot naturally handle temporal aspects of flight data
    - Struggles with variable-density clusters of flight patterns
    - No inherent mechanism for anomaly detection
- Isolation Forest
  - Pros:
  - Specifically designed for anomaly detection
    - Computationally efficient even with large datasets
    - Works well with high-dimensional flight data
  - Cons:
    - No clustering capability for normal flight patterns
    - Cannot capture temporal relationships in data
    - Limited interpretability of why something is classified as anomalous

Probabilistic Models 

- Hidden Markov Models (HMMs)
  - Pros:
    - Naturally models flight phase transitions
    - Computationally efficient once trained
    - Very interpretable state transitions
  - Cons:
    - Struggles with continuous, high-dimensional flight data
    - Limited ability to detect subtle anomalies
    - Can be biased by initial state assumptions

Deep Learning Models

- Autoencoder + HDBSCAN
  - Pros:
    - Simultaneously learns normal flight patterns while providing rich anomaly detection through reconstruction error
    - Produces interpretable latent space representations that help explain both clusters and anomalies
    - Can detect both global pattern deviations and local anomalies (like sudden accelerations) in a single model
  - Cons:
    - Requires significant tuning of both autoencoder architecture and HDBSCAN parameters
    - May overfit to common flight patterns if training data isn't sufficiently diverse
    - Computationally intensive during training phase compared to traditional methods

- GANs
  - Pros:
    - Can learn very subtle deviations from normal flight patterns
    - Particularly good at handling multimodal distributions in flight behaviors
    - Can generate synthetic examples to help validate anomaly detection
  - Cons:
    - Notoriously difficult to train and achieve stable convergence
  - Much higher computational overhead than other approaches
    - May suffer from mode collapse, missing important but rare flight patterns

- LSTM-based approaches
  - Pros:
    - Naturally handles temporal dependencies in flight patterns
    - Excellent at predicting expected behavior sequences
    - Can capture long-term dependencies in flight phases
  - Cons:
    - Tends to focus on global patterns, potentially missing local anomalies
    - Memory requirements scale with sequence length
    - Can be biased toward more common flight patterns, potentially normalizing subtle anomalies

Although it would probably be interesting to apply all of the models listed above, the becase of the scope of this project (and the fact that I sank so much time extracting and trandorming the data) we are only gong to choose one... and we're going with the Autoencoder + HDBSCAN combination because it uniquely provides both rich anomaly detection capabilities through reconstruction error and effective clustering of normal flight patterns in a single model architecture, while maintaining interpretability through its latent space representations. This approach outperforms other models by simultaneously capturing both global flight patterns and local anomalies (such as unusual accelerations or trajectory deviations), and though it requires more computational resources during training, the quality of insights it provides justifies this tradeoff compared to simpler approaches.

### Tools used

Before going into the individual models, let's take a look at the core packages doing the work here and talk about why we chose them.

- Core Machine Learning Tools:
  - PyTorch:
    - Implements autoencoder architecture for dimensionality reduction of flight data
    - Provides GPU acceleration for training through CUDA integration
    - Manages batch processing and gradient optimization through DataLoader
    - Generally easier implementation and tuning than Tensorflow for simliar models

  - HDBSCAN:
    - Performs density-based clustering to identify distinct flight patterns
    - Handles noise points and variable-density clusters automatically
    - Determines optimal number of clusters through hierarchical density estimation

  - Scikit-learn:
    - Standardizes features through robust scaling operations
    - Removes outliers using statistical methods (z-score analysis)
    - Provides consistent preprocessing pipeline for both training and inference

- Data Management Tools:
  - SQLAlchemy + psycopg2:
    - Handles efficient batch retrieval of flight telemetry data
    - Maintains connection pooling for concurrent database operations
    - Provides ORM interface for complex flight data queries

  - Pandas:
    - Processes time-series flight data through vectorized operations
    - Manages complex feature engineering for altitude and speed metrics
    - Handles missing data and temporal alignment of flight records

In [1]:
# import dependencies 
import pandas as pd
from IPython.display import display, HTML  
import IPython.core.getipython as getipython
import json
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import torch
from torch.serialization import add_safe_globals

# import model object
sys.path.append('../scripts')
from _09_autoencoder_training import Autoencoder

# Add Autoencoder to safe globals
add_safe_globals([Autoencoder])

### Autoencoder

#### Hyperparameter Selection

- Input Structure (450 dimensions)
  - Represents 9 flight features sampled at 50 timepoints
  - Creates temporal sequence for pattern learning
  - Requires deeper architecture versus point-in-time features

- Network Width (256 → 128 → 64 → 32)
  - Progressive quartering preserves temporal relationships
  - Width accommodates complex time-series patterns
  - Prevents information bottlenecks across 450-dimensional input

- ReLU Activation Functions
  - Mitigates vanishing gradients across deep architecture
  - Enhances learning of temporal patterns
  - Maintains non-linearity without sigmoid saturation issues

- Encoding Dimension (32)
  - Implements ~14:1 compression ratio from 450 input features
  - Maintains capacity for temporal pattern variations
  - Balances compression against reconstruction accuracy

- Batch Size (32)
  - Ensures stable gradient updates for temporal sequences
  - Optimizes 32GB memory constraint
  - Maintains temporal diversity within batches

- Training Parameters
  - 100k random samples from full dataset
  - 60-minute training window with GPU acceleration
  - Memory-constrained sample size enables efficient iteration

- Training Convergence
  - Deeper architecture requires longer training vs previous version
  - 60-minute window allows sufficient epochs for temporal pattern learning
  - Memory constraints influence convergence trade-offs

In [2]:
# Load the model
model_path = '../data/_09_autoencoder_training/model.pth'
loaded_autoencoder = torch.load(model_path, map_location=torch.device('cpu'), weights_only=False)
loaded_autoencoder.eval()

print("=== Basic Model Information ===")
print(f"Model Type: {type(loaded_autoencoder).__name__}")
print(f"Model Mode: {'Training' if loaded_autoencoder.training else 'Evaluation'}")

print("\n=== Architecture Overview ===")
print("Encoder Architecture:")
print(loaded_autoencoder.encoder)
print("\nDecoder Architecture:")
print(loaded_autoencoder.decoder)

print("\n=== Dimensionality Analysis ===")
# Get input/encoding dimensions from the first and last layer of encoder
input_dim = next(loaded_autoencoder.encoder.children()).in_features
encoding_dim = list(loaded_autoencoder.encoder.children())[-2].out_features  # -2 to skip ReLU
print(f"Input Dimension: {input_dim}")
print(f"Encoding Dimension: {encoding_dim}")
print(f"Compression Ratio: {input_dim/encoding_dim:.1f}x")

print("\n=== Parameter Analysis ===")
# Separate encoder and decoder parameters
encoder_params = sum(p.numel() for p in loaded_autoencoder.encoder.parameters())
decoder_params = sum(p.numel() for p in loaded_autoencoder.decoder.parameters())
total_params = encoder_params + decoder_params

print(f"Encoder Parameters: {encoder_params:,}")
print(f"Decoder Parameters: {decoder_params:,}")
print(f"Total Parameters: {total_params:,}")

print("\n=== Layer Details ===")
for name, param in loaded_autoencoder.state_dict().items():
    if isinstance(param, torch.Tensor):
        print(f"\nLayer: {name}")
        print(f"Shape: {list(param.size())}")
        print(f"Parameters: {param.numel():,}")
        print(f"Data Type: {param.dtype}")
        # Add basic statistics for weight matrices
        if 'weight' in name:
            print(f"Weight Stats:")
            print(f"  Mean: {param.mean().item():.4f}")
            print(f"  Std: {param.std().item():.4f}")
            print(f"  Min: {param.min().item():.4f}")
            print(f"  Max: {param.max().item():.4f}")

print("\n=== Hardware Information ===")
devices = set(param.device for param in loaded_autoencoder.parameters())
print(f"Model Device(s): {devices}")

=== Basic Model Information ===
Model Type: Autoencoder
Model Mode: Evaluation

=== Architecture Overview ===
Encoder Architecture:
Sequential(
  (0): Linear(in_features=450, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): ReLU()
  (6): Linear(in_features=64, out_features=32, bias=True)
  (7): ReLU()
)

Decoder Architecture:
Sequential(
  (0): Linear(in_features=32, out_features=64, bias=True)
  (1): ReLU()
  (2): Linear(in_features=64, out_features=128, bias=True)
  (3): ReLU()
  (4): Linear(in_features=128, out_features=256, bias=True)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=450, bias=True)
)

=== Dimensionality Analysis ===
Input Dimension: 450
Encoding Dimension: 32
Compression Ratio: 14.1x

=== Parameter Analysis ===
Encoder Parameters: 158,688
Decoder Parameters: 159,106
Total Parameters: 317,794

=== Layer Details ===

Layer: en

#### Model output data

Now lets look at the actual outputs from the autoencoder and go over what they mean.

In [3]:
# Examine outputs
df = pd.read_parquet('../data/_11_analysis_conclusions/autoencoder_output_sample.parquet')

print(f"number of rows: {len(df)}")

display(df.head())

number of rows: 100


,time_offset_0,vertical_rate_0,ground_speed_0,heading_0,altitude_0,vertical_accel_0,ground_accel_0,turn_rate_0,climb_descent_accel_0,time_offset_1,...,time_offset_49,vertical_rate_49,ground_speed_49,heading_49,altitude_49,vertical_accel_49,ground_accel_49,turn_rate_49,climb_descent_accel_49,segment_id
987231,0.007142,50.109253,546.487183,94.370583,34493.812500,-4.301480,-0.116590,0.684139,-31.687557,17424902.0,...,765742720.0,-51.899387,547.978638,72.161011,35430.289062,57.352501,0.024926,-0.333013,-81.800285,8592900
79954,0.003918,82.439018,460.512634,217.097153,35535.257812,6.474170,0.088270,0.007591,20.345068,7059239.0,...,639985984.0,-110.646332,464.905518,210.165787,35595.960938,-3.423635,-0.129447,-0.084875,-43.646847,9125411
567130,0.007740,637.542297,91.083076,95.346695,245.131027,2.802289,-0.533764,-0.150562,-36.902733,5696532.5,...,463879424.0,91.232872,109.719849,53.798630,3160.060547,16.056448,0.228994,-0.235181,-127.201141,11826840
500891,0.006670,-48.454681,414.664429,270.932648,38325.113281,-25.247885,-0.099923,0.151855,4.844756,18432414.0,...,869229888.0,-95.528175,412.022003,283.591400,38207.257812,-20.732584,-0.146490,0.031765,-38.067490,8862811
55399,0.026402,161.986526,116.810905,62.545231,3702.970947,5.630402,-0.747427,0.962443,-101.030655,4920148.5,...,190356688.0,135.428925,119.363556,238.775909,4430.441406,-74.548874,-0.829505,0.026354,-86.863358,10483710


- Understanding Autoencoder Output
  - The output is a "reconstruction" of the input data
  - Think of it like a very sophisticated game of "telephone":
    - The autoencoder first compresses the flight data (encoder)
    - Then tries to recreate the original data from the compressed version (decoder)
    - The output shows how well it recreated each detail
  - Reconstruction error measures the difference between:
    - What went in (original flight pattern)
    - What came out (reconstructed flight pattern)
  - Large differences (high reconstruction error) suggest:
    - Unusual flight patterns
    - Potential anomalies
    - Patterns the model hasn't learned well
  - Small differences (low reconstruction error) suggest:
    - Common flight patterns
    - Normal behavior
    - Patterns similar to training data

- Input Structure (450 dimensions)
  - 9 flight parameters: time_offset, vertical_rate, ground_speed, heading, altitude, vertical_accel, ground_accel, turn_rate, climb_descent_accel
  - Each parameter sampled across 50 sequential points (9 × 50 = 450 dimensions) 
  - Creates temporal sequence for learning flight trajectory patterns
  - Input preserves chronological relationships in flight behavior

- Data Preprocessing
  - Input data drawn from global ADS-B flight traces
  - Continuous traces segmented into 50-point sequences
  - Each sequence represents a flight behavior pattern
  - Parameters maintain original units and scaling
 
- Architecture Purpose 
  - Pattern Recognition: Learn common flight trajectory patterns
  - Anomaly Detection: Identify unusual flight behaviors
  - Dimensionality Reduction: Compress 450D input into meaningful representation
  - Sequence Analysis: Capture temporal patterns in flight data

- Output Structure
  - Matches input dimensionality (450D)
  - Reconstructs full sequence of 50 timepoints
  - Preserves all 9 original flight parameters
  - Enables comparison between input and reconstructed patterns

- Training Approach
  - Uses full day of global flight data
  - Learns from diverse flight patterns and behaviors
  - Captures normal variations in flight trajectories
  - Establishes baseline for anomaly detection

### HDBSCAN

#### Hyperparameters Selection

- Min Cluster Size (35)
  - Started at 100 (0.1% of original dataset)
  - Experimented with extremes (10-100 range)
  - Found sweet spot at 35 for 10k sample size
  - Large enough to ensure statistical significance
  - Small enough to capture distinct flight patterns
  - Balances between over-fragmentation (87 clusters) and under-segmentation (3 clusters)
  - Aligned with visible trajectory groupings in visualization
 
- Min Samples (4)
  - Started conservative at 5
  - Tested range from 2 (too fragmented) to 8 (too merged)
  - Provides sufficient statistical basis for core points
  - Allows detection of natural trajectory groupings
  - Controls noise without overly restricting cluster formation
  - Works well with normalized high-dimensional flight data

- Epsilon (0.25)
  - Started conservative at 0.1
  - Experimented between 0.1 and 0.4
  - Accommodates spatial spread visible in 3D trajectory plots
  - Handles normalized data in high-dimensional space
  - Connects related trajectories without over-merging
  - Balances cluster expansion with pattern distinction

- Cluster Selection Method ('eom')
  - Maintained 'eom' throughout process
  - Well-suited for varying density clusters
  - Handles flight patterns of different densities
  - More robust to noise than alternative methods
  - Provides stable cluster selection

- Distance Metric (Euclidean)
  - Natural choice for normalized continuous flight data
  - Works well in high-dimensional spaces
  - Computationally efficient for large datasets
  - Appropriate for spatial trajectory patterns

##### Supporting Decisions

- Data Preprocessing
  - StandardScaler normalization
  - Outlier removal using z-score method (3 standard deviations)
  - 10k sample size for computational efficiency
  - Input data: 450 dimensions (9 flight parameters × 50 timepoints)

- Tuning Process
  - Visualization-driven iteration
  - Started conservative, tested extremes
  - Found balance between cluster count and noise
  - Used domain knowledge of flight patterns
  - Iterative refinement based on trajectory visualization

In [4]:
import pickle
import numpy as np

# Load the pickled model
with open('../data/_10_clustering/model.pkl', 'rb') as f:
    clusterer = pickle.load(f)

# Basic model info
print(f"Model Type: {type(clusterer).__name__}")
print(f"Min Cluster Size: {clusterer.min_cluster_size}")
print(f"Cluster Selection Method: {clusterer.cluster_selection_method}")

# Clustering results
n_clusters = len(np.unique(clusterer.labels_)) - (1 if -1 in clusterer.labels_ else 0)
print(f"\nNumber of clusters: {n_clusters}")
print(f"Number of noise points: {np.sum(clusterer.labels_ == -1)}")
print(f"Total points: {len(clusterer.labels_)}")

# Cluster sizes
for label in np.unique(clusterer.labels_[clusterer.labels_ != -1]):
    print(f"\nCluster {label} size: {np.sum(clusterer.labels_ == label)}")

Model Type: HDBSCAN
Min Cluster Size: 35
Cluster Selection Method: eom

Number of clusters: 10
Number of noise points: 4916
Total points: 10000

Cluster 0 size: 51

Cluster 1 size: 35

Cluster 2 size: 46

Cluster 3 size: 101

Cluster 4 size: 2803

Cluster 5 size: 56

Cluster 6 size: 35

Cluster 7 size: 41

Cluster 8 size: 138

Cluster 9 size: 1778


In [5]:
#### Examine outputs

# read in sample data
df = pd.read_parquet('../data/_10_clustering/results.parquet').set_index('segment_id')

print(f"number of rows: {len(df)}")

display(df.head())


number of rows: 10000


,cluster,is_most_typical,is_most_extreme
segment_id,,,
8592900,5,False,False
9125411,-1,False,False
11826840,4,False,False
8862811,8,False,False
10483710,-1,False,False


The outputs of our cluster model are much more intutive to interpret than the autoencoder output. For each segment ID we get a cluster label. It is actually possible to generate a ton of cool metrics using the clusters themselves. By looking at the number of points in a cluster, the centrality of 1 point with regard to the cluster centroid, and many more insights. But that is out of scope for today. 

What we are calculating, is the most "typical" and the most "extreme" trace within each cluster, and storing it to a boolean value. We define the calcualtions we use and the visualize the most typical and extreme traces in the next notebook. 

#### Constraints

Up until this point I have been able to get over all compute and time restraints by utiliziation tools like connection pooling and parellelization. But with these models I have hit a will. For the autoencoder with GPU acceleration, I was able only able to process 100k rows before hitting system memory contraints. That said, running 45 million data pionts through a nueral network isn't that bad.

For my HDSBCAN model, I am currently only able to do a sample of 25k rows. Trying to do a sample of 50k rows, it ran for 8 hours and still didn't complete. I was not able to get the GPU acceleration properly working for HDBSCAN due to dependancy issues.

I will discuss more about what I woudl do in the future to resolve these constratints and increase my sample size at the end of the next notebook where I talk about ideas for futrue anaylsis.